In [14]:
import pandas as pd
import numpy as np
import os
import csv
import itertools
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from tqdm.notebook import tqdm
from hyperopt import tpe,hp,Trials
from hyperopt.fmin import fmin
import math
import optuna


In [15]:
repo_pool = []
for filename in os.listdir('./data_use/'):
    repo_pool.append(os.path.join(filename))
    
base_df = pd.read_csv('./data_use/' + repo_pool[0])
base_df = base_df.drop(columns=['dates'])
print(base_df.columns)

Index(['monthly_commits', 'monthly_contributors', 'monthly_open_PRs',
       'monthly_closed_PRs', 'monthly_merged_PRs', 'monthly_PR_mergers',
       'monthly_PR_comments', 'monthly_open_issues', 'monthly_closed_issues',
       'monthly_issue_comments', 'monthly_stargazer', 'monthly_forks',
       'monthly_watchers'],
      dtype='object')


In [16]:
repo_pool = []
for filename in os.listdir('./data_use/'):
    repo_pool.append(os.path.join(filename))
    
base_df = pd.read_csv('./data_use/' + repo_pool[0])
base_df = base_df.drop(columns=['dates'])
target_col = 'monthly_commits_12mo'
real_df_cols = [col for col in base_df.columns]
real_df_cols.append(target_col)
lines = []
projects = []
train_lines = []
test_lines = []
p_names = os.listdir('./data_use/')
#print(repo_pool)
for index, repo in enumerate(repo_pool):
    print(repo)
    project = []
    p_train_lines = []
    p_test_lines = []
    df = pd.read_csv('./data_use/' + repo)
    df = df.drop(columns=['dates'])

    matrix = df.to_numpy()
    for i in range(matrix.shape[0]-12):
        row = []
        for j in range(matrix.shape[1]):
            row.append(matrix[i][j])
        row.append(matrix[i+12][0])
        lines.append(row)
    for i, line in enumerate(lines):
        if i < len(lines)*0.7:
            train_lines.append(line)
            p_train_lines.append(line)
        else:
            test_lines.append(line)
            p_test_lines.append(line)
    print('./datasets/health_'+p_names[index][:-4])
#     p_df = pd.DataFrame(p_train_lines, columns = real_df_cols)
#     p_df.to_csv('./datasets/'+target_col+'/health_'+p_names[index][:-4]+'_train.csv', index=False)
#     p_df_t = pd.DataFrame(p_test_lines, columns = real_df_cols)
#     p_df.to_csv('./datasets/'+target_col+'/health_'+p_names[index][:-4]+'_test.csv', index=False)
    project.append(p_train_lines)
    project.append(p_test_lines)
    projects.append(project)
    lines = []

project0000.csv
./datasets/health_project0000
project0001.csv
./datasets/health_project0001
project0002.csv
./datasets/health_project0002
project0003.csv
./datasets/health_project0003
project0004.csv
./datasets/health_project0004
project0005.csv
./datasets/health_project0005
project0006.csv
./datasets/health_project0006
project0007.csv
./datasets/health_project0007
project0008.csv
./datasets/health_project0008
project0009.csv
./datasets/health_project0009
project0010.csv
./datasets/health_project0010
project0011.csv
./datasets/health_project0011


In [17]:
train_df = pd.DataFrame(train_lines, columns = real_df_cols)

In [18]:
test_df = pd.DataFrame(test_lines, columns = real_df_cols)

In [19]:
test_df.head()

,monthly_commits,monthly_contributors,monthly_open_PRs,monthly_closed_PRs,monthly_merged_PRs,monthly_PR_mergers,monthly_PR_comments,monthly_open_issues,monthly_closed_issues,monthly_issue_comments,monthly_stargazer,monthly_forks,monthly_watchers,monthly_closed_PRs_12mo
0,214,6,14,0,12,2,13,3,42,24,194,27,0,2
1,232,16,17,1,18,2,29,12,92,113,201,30,1,4
2,121,12,17,3,14,1,34,6,38,84,254,23,1,0
3,301,12,20,2,17,3,50,15,88,192,99,22,0,3
4,222,10,17,3,14,2,33,9,57,104,163,32,0,1


In [29]:
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 200)
    max_depth = trial.suggest_int("max_depth", 5, 20)
    criterion_enum = trial.suggest_int("criterion", 1, 2)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5)
    min_impurity_decrease = trial.suggest_float("min_impurity_decrease", 0, 10)
    
    criterion = 'squared_error'
    if criterion_enum == 0:
            criterion = 'squared_error'
    elif criterion_enum == 1:
        criterion = 'absolute_error'
    model = RandomForestRegressor(n_estimators=n_estimators, criterion=criterion, min_samples_leaf=min_samples_leaf, min_impurity_decrease=min_impurity_decrease, max_depth=max_depth)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    mmre_l = []
    mmreg_l = []
    pred40 = 0
    for y_t, y_p in zip(y_test, y_pred):
        num = np.abs(y_p - y_t)
        den = np.abs(y_t)
        mre = 0
        mreg = 0
        if den == 0:
            if num != 0:
                den+=1
                num+=1
                mre = num/den
                mmre_l.append(mre)
            else:
                mmre_l.append(mre)
        else:
            mre = num/den
            mmre_l.append(mre)
        if mre <= 0.4:
            pred40+=1
        
    MRE = np.median(np.array(mmre_l))
    MREG = np.median(np.array(mmreg_l))
    limit = int(len(y_test)*.7)
    sa_num = mean_absolute_error(y_test,  y_pred)
    y_predg = np.nan_to_num([(np.median(y_pred[:i])) for i in range(len(y_pred))])
    se_den = mean_absolute_error(y_test, y_predg)
    if se_den == 0:
        if sa_num == 0:
            acc = 1
        acc = 1 - ( (sa_num + 1) / (se_den+1) )
    else:
        acc = 1 - ( sa_num / se_den )
        
    return MRE, pred40, acc


In [28]:
import warnings
import time
warnings.filterwarnings("ignore")
evals = []
model_data_l = []
index=0
for project in projects:
    eval_row , data_row = [], []
    start_time = time.time()
    global mre_list
    mre_list = [1]
    global pred40_list
    pred40_list = [0]
    global acc_list
    acc_list = [0]
    # pbar.set_description('Running %s' %p_names[index])
    metrics_data = []
    train_df = pd.DataFrame(project[0], columns = real_df_cols)
    test_df = pd.DataFrame(project[1], columns = real_df_cols)
    X_train = train_df.iloc[:, 1:-1]
    X_test = test_df.iloc[:, 1:-1]
    y_train = train_df.iloc[:, -1:].values.flatten().tolist()
    y_test = np.array(test_df.iloc[:, -1:].values.flatten().tolist())
    mre_past = [0]
    pred40_past = [1]
    acc_past = [1]
    study = optuna.create_study(directions=["minimize", "maximize", "maximize"])
    study.optimize(objective, n_trials=100)
    print(study.best_trials[-1].values)
    params = study.best_trials[-1].params
    data_row.append(p_names[index])
    data_row.append('optuna')
    data_row.append(3500)
    criterion = params['criterion']
    n_estimators=int(params['n_estimators'])
    min_samples_leaf=int(params['min_samples_leaf'])
    min_impurity_decrease=params['min_impurity_decrease']
    max_depth=int(params['max_depth'])
    data_row.append(n_estimators)
    data_row.append(criterion)
    data_row.append(min_samples_leaf)
    data_row.append(min_impurity_decrease)
    data_row.append(max_depth)
    eval_row.append(round(study.best_trials[-1].values[0] * 100, 2))
    eval_row.append(study.best_trials[-1].values[1])
    eval_row.append(round(study.best_trials[-1].values[2]*100, 2))
    data_row.append(time.time() - start_time)
    evals.append(eval_row)
    model_data_l.append(data_row)
   
    index+=1


[I 2022-05-05 16:48:20,340] A new study created in memory with name: no-name-24decec0-9423-4bf8-b990-4ff0bcd936dd
[I 2022-05-05 16:48:20,351] Trial 0 finished with values: [0.6309523809523809, 1.0, 0.132665832290363] and parameters: {'n_estimators': 12, 'max_depth': 6, 'criterion': 2, 'min_samples_leaf': 4, 'min_impurity_decrease': 5.537564383681312}. 
[I 2022-05-05 16:48:20,446] Trial 1 finished with values: [0.5814246894409938, 1.0, 0.12353850912465214] and parameters: {'n_estimators': 184, 'max_depth': 7, 'criterion': 2, 'min_samples_leaf': 5, 'min_impurity_decrease': 8.3222812835651}. 
[I 2022-05-05 16:48:20,546] Trial 2 finished with values: [0.563533834586466, 1.0, 0.12019404241671006] and parameters: {'n_estimators': 190, 'max_depth': 11, 'criterion': 2, 'min_samples_leaf': 4, 'min_impurity_decrease': 3.7383681636505592}. 
[I 2022-05-05 16:48:20,622] Trial 3 finished with values: [0.5838293650793649, 1.0, 0.12398609501738123] and parameters: {'n_estimators': 144, 'max_depth': 7,

[I 2022-05-05 16:48:22,474] Trial 32 finished with values: [0.6461748633879781, 2.0, 0.09932885906040267] and parameters: {'n_estimators': 61, 'max_depth': 11, 'criterion': 1, 'min_samples_leaf': 4, 'min_impurity_decrease': 7.358369587225515}. 
[I 2022-05-05 16:48:22,573] Trial 33 finished with values: [0.6493445692883895, 2.0, 0.0985267034990791] and parameters: {'n_estimators': 178, 'max_depth': 19, 'criterion': 1, 'min_samples_leaf': 5, 'min_impurity_decrease': 7.53374095035424}. 
[I 2022-05-05 16:48:22,658] Trial 34 finished with values: [1.1294964028776977, 3.0, 0.06581185398403766] and parameters: {'n_estimators': 139, 'max_depth': 18, 'criterion': 1, 'min_samples_leaf': 2, 'min_impurity_decrease': 0.08567443279096865}. 
[I 2022-05-05 16:48:22,689] Trial 35 finished with values: [0.6524822695035462, 1.0, 0.0977312390924957] and parameters: {'n_estimators': 47, 'max_depth': 5, 'criterion': 1, 'min_samples_leaf': 4, 'min_impurity_decrease': 5.915768688401577}. 
[I 2022-05-05 16:48:

[I 2022-05-05 16:48:24,256] Trial 64 finished with values: [0.5677083333333333, 1.0, 0.11874105865522178] and parameters: {'n_estimators': 80, 'max_depth': 11, 'criterion': 2, 'min_samples_leaf': 1, 'min_impurity_decrease': 5.326733443466679}. 
[I 2022-05-05 16:48:24,330] Trial 65 finished with values: [0.6243686868686869, 2.0, 0.10480887792848348] and parameters: {'n_estimators': 132, 'max_depth': 5, 'criterion': 1, 'min_samples_leaf': 4, 'min_impurity_decrease': 7.122910021359161}. 
[I 2022-05-05 16:48:24,383] Trial 66 finished with values: [0.564868804664723, 1.0, 0.12044447898049582] and parameters: {'n_estimators': 98, 'max_depth': 16, 'criterion': 2, 'min_samples_leaf': 5, 'min_impurity_decrease': 7.130762713894956}. 
[I 2022-05-05 16:48:24,444] Trial 67 finished with values: [0.5866815249967424, 1.0, 0.12451637953122219] and parameters: {'n_estimators': 115, 'max_depth': 11, 'criterion': 2, 'min_samples_leaf': 4, 'min_impurity_decrease': 1.9351443197756402}. 
[I 2022-05-05 16:48

[I 2022-05-05 16:48:26,458] Trial 96 finished with values: [0.6158967391304346, 1.0, 0.12991144355601403] and parameters: {'n_estimators': 115, 'max_depth': 11, 'criterion': 2, 'min_samples_leaf': 4, 'min_impurity_decrease': 1.9351443197756402}. 
[I 2022-05-05 16:48:26,515] Trial 97 finished with values: [0.5806122448979594, 1.0, 0.12338718421337969] and parameters: {'n_estimators': 105, 'max_depth': 6, 'criterion': 2, 'min_samples_leaf': 4, 'min_impurity_decrease': 9.618888911630263}. 
[I 2022-05-05 16:48:26,540] Trial 98 finished with values: [0.5694444444444443, 1.0, 0.11832061068702304] and parameters: {'n_estimators': 41, 'max_depth': 19, 'criterion': 2, 'min_samples_leaf': 5, 'min_impurity_decrease': 6.849682895689877}. 
[I 2022-05-05 16:48:26,649] Trial 99 finished with values: [0.6403281519861831, 2.0, 0.10080474375264725] and parameters: {'n_estimators': 193, 'max_depth': 17, 'criterion': 1, 'min_samples_leaf': 5, 'min_impurity_decrease': 8.34186941662656}. 
[I 2022-05-05 16:4

[0.5806122448979594, 1.0, 0.12338718421337969]


[I 2022-05-05 16:48:27,027] Trial 2 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 176, 'max_depth': 16, 'criterion': 2, 'min_samples_leaf': 3, 'min_impurity_decrease': 8.877226140533862}. 
[I 2022-05-05 16:48:27,091] Trial 3 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 109, 'max_depth': 14, 'criterion': 1, 'min_samples_leaf': 4, 'min_impurity_decrease': 1.9318923246672604}. 
[I 2022-05-05 16:48:27,206] Trial 4 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 200, 'max_depth': 5, 'criterion': 1, 'min_samples_leaf': 2, 'min_impurity_decrease': 6.798786658080998}. 
[I 2022-05-05 16:48:27,237] Trial 5 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 53, 'max_depth': 16, 'criterion': 2, 'min_samples_leaf': 4, 'min_impurity_decrease': 6.119718188441697}. 
[I 2022-05-05 16:48:27,289] Trial 6 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 90, 'max_depth': 10, 'criterion': 2, '

[I 2022-05-05 16:48:29,337] Trial 39 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 108, 'max_depth': 13, 'criterion': 1, 'min_samples_leaf': 2, 'min_impurity_decrease': 7.805308909802852}. 
[I 2022-05-05 16:48:29,366] Trial 40 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 51, 'max_depth': 13, 'criterion': 2, 'min_samples_leaf': 5, 'min_impurity_decrease': 3.2550331980270286}. 
[I 2022-05-05 16:48:29,419] Trial 41 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 102, 'max_depth': 19, 'criterion': 2, 'min_samples_leaf': 3, 'min_impurity_decrease': 2.0142669355070133}. 
[I 2022-05-05 16:48:29,458] Trial 42 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 68, 'max_depth': 16, 'criterion': 2, 'min_samples_leaf': 1, 'min_impurity_decrease': 1.8632109184395995}. 
[I 2022-05-05 16:48:29,524] Trial 43 finished with values: [0.0, 8.0, 0.0] and parameters: {'n_estimators': 123, 'max_depth': 17, 'criterio

KeyboardInterrupt: 

In [49]:
evals_data = pd.DataFrame(evals, columns = ['MdMRE', 'PRED40', 'SA']).fillna(0)

In [50]:
model_data = pd.DataFrame(model_data_l, columns = ['ds', 'Optimizer', 'Models built','N_estimators', 'Criterion', 'Min_samples_leaf','Min_impurity_decrease', 'Max_depth', 'Time']).fillna(0)

In [51]:
eval_data_full = pd.concat([model_data, evals_data], axis=1)

In [52]:
eval_data_full = eval_data_full.fillna(0)

In [53]:
normalized_eval_data=(evals_data-evals_data.min())/(evals_data.max()-evals_data.min())


In [54]:
normalized_eval_data.columns = ['N_MdMRE', 'N_PRED40', 'N_SA']

In [55]:
eval_data_full = pd.concat([normalized_eval_data, evals_data], axis=1)

In [56]:
eval_data_full.head()

,N_MdMRE,N_PRED40,N_SA,MdMRE,PRED40,SA
0,0.279994,0.090909,0.928886,60.00,1,12.70
1,0.000000,0.727273,0.842790,0.00,8,0.00
2,0.500257,0.000000,0.781167,107.20,0,-9.09
3,0.540063,0.000000,0.815335,115.73,0,-4.05
4,0.735499,0.000000,0.803268,157.61,0,-5.83


In [57]:
all_data_full = pd.concat([eval_data_full, model_data], axis=1)

In [58]:
all_data_full.to_csv('./evals/'+target_col+'/optuna_all_data_full_multi.csv', index=False)